<a href="https://colab.research.google.com/github/alienverarslan/Movie_Recommender_System/blob/main/User_to_User_CollaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [211]:
import pandas as pd
import numpy as np
from sklearn import neighbors as nei
from sklearn import metrics as met

In [67]:
data = pd.read_csv("train.csv")
data.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [ ]:
data.describe()

,userID,movieID,rating
count,990209.000000,990209.00000,990209.000000
mean,3016.430061,1865.56350,3.581463
std,1728.237347,1096.00108,1.117173
min,1.000000,1.00000,1.000000
25%,1565.000000,1030.00000,3.000000
50%,2970.000000,1835.00000,4.000000
75%,4535.000000,2770.00000,4.000000
max,6040.000000,3952.00000,5.000000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990209 entries, 0 to 990208
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userID   990209 non-null  int64
 1   movieID  990209 non-null  int64
 2   rating   990209 non-null  int64
dtypes: int64(3)
memory usage: 22.7 MB


In [68]:
#Define the Utility Matrix

U = data.pivot("userID", "movieID", "rating")
U.head()

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,3.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,3.0,NaN,NaN,2.0,4.0,3.0,NaN,4.0,3.0,4.0,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
train_share = 0.80
train_size = int(U.shape[0]*train_share)
train = U[:train_size]
test = U[train_size:]

In [214]:
#Take 5 movies randomly for users from test set 

sampleTest = test.sample(100, random_state=42)

samplelist = []
for i in sampleTest.index:
  v = sampleTest.loc[i: i,:].dropna(axis=1)
  ss = np.random.choice(v.columns, 5)
  samplelist.append([i, list(ss)])

In [186]:
def CollaborativeFilteringUtoU(train_df, test_df, test_sample, k=20, met='minkowski', avg='uniform'):
  myPreds = []
  realLabels = []
  for sample in test_sample:
    userId = sample[0]
    for movieId in sample[1]:
      filtered_train = train_df.dropna(subset=[movieId])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
             
      if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k, metric=met, n_jobs=-1)
        nn.fit(filtered_train)

        test_row = test_df.loc[userId:userId, :].copy()
        test_row[movieId] = np.nan
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        distances, indices = nn.kneighbors(test_row, return_distance=True)

        userIds = filtered_train.iloc[indices[0], :].index
        if avg == 'weighted':
          avgs = np.average(filtered_train.loc[userIds, movieId].values, weights=(1 - distances[0]+0.000001))
        elif avg == 'uniform':
          avgs = filtered_train.loc[userIds, movieId].values.mean() + 0.000001


        pred = np.nanmean(test_df.loc[userId,:].values) + 0.000001 + avgs

        if pred > 5:
          pred = 5
        if pred < 1:
          pred = 1
      
        myPreds.append(pred)

        real_label = test_df.loc[userId, movieId]
        realLabels.append(real_label)
  
  return [realLabels, myPreds] 
  

In [215]:
#Finding the optimal parameters 
metrics = ['cosine']
averages = ['uniform', 'weighted']
k_size = [10, 20, 30, 40, 50]
params = []
for m in metrics:
  for avg in averages:
      for k in k_size:
        params.append([m, avg, k])

print("params                    rmse")
results = []
for i in range(len(params)):
  labels, preds = CollaborativeFilteringUtoU(train_df=train, test_df=test, test_sample=samplelist, met=params[i][0], avg=params[i][1], k=params[i][2])
  results.append(met.mean_squared_error(labels, preds, squared=False))  
  print(params[i], results[i])

params                    rmse
['cosine', 'uniform', 10] 0.9218036180042753
['cosine', 'uniform', 20] 0.9127945529345801
['cosine', 'uniform', 30] 0.9194404770176241
['cosine', 'uniform', 40] 0.917851950593896
['cosine', 'uniform', 50] 0.918766592454607
['cosine', 'weighted', 10] 0.9158569600612906
['cosine', 'weighted', 20] 0.9070730672832742
['cosine', 'weighted', 30] 0.9109146843791628
['cosine', 'weighted', 40] 0.9081270211742599
['cosine', 'weighted', 50] 0.9236733147965277


In [216]:
best_params = params[np.argmin(results)]
best_score = results[np.argmin(results)]
best_params

['cosine', 'weighted', 20]

In [226]:
testlist = []
for i in test.index:
  v = test.loc[i: i,:].dropna(axis=1)
  ss = np.random.choice(v.columns, 10)
  testlist.append([i, list(ss)])


labels, preds = CollaborativeFilteringUtoU(train_df=train, test_df=test, test_sample=testlist[:600], met=best_params[0], avg=best_params[1], k=best_params[2])
test_rmse = met.mean_squared_error(labels, preds, squared=False)
print("Test RMSE is :", test_rmse)

Test RMSE is : 0.9154684789366111
